In [228]:
import pandas as pd
import numpy as np

df = pd.read_excel (r'D:\Crocs Source Code\Prototype 1\mold_table.xlsx')

In [229]:
#!pip install ortools

from ortools.linear_solver import pywraplp

In [230]:
df.head()

,Asset_serial_number,Number_of_pairs,Set,Size,Mold_number,Injection_volume,Injection_time,Pulling_time,Curing_time,Operation_time,Mold_name,Cycle_Time,Unnamed: 12,Unnamed: 13,Change Color,25,Unnamed: 16
0,57560,2,Set T,C4,IJ,195.0,13,30,360,120,C4IJ,493,NaN,NaN,Mold Change,30,NaN
1,60301,2,Set R1,C4,UV,197.0,13,30,360,130,C4UV,503,NaN,NaN,NaN,NaN,NaN
2,64460,2,Set R2,C4,K1L1,197.0,13,30,360,120,C4K1L1,493,NaN,NaN,NaN,NaN,NaN
3,66646,4,Set B3,C4,MNOP,292.7,19,30,360,90,C4MNOP,469,NaN,NaN,1 máy,1 style,?
4,57561,2,Set T,C5,KL,218.0,14,30,360,120,C5KL,494,NaN,NaN,NaN,NaN,NaN


In [231]:
# Step 1: Update demand by style, color, size and quantity

# Get demand info

size = pd.Series(['C4','C5','C7', 'J2', 'C8'])
quantity = pd.Series([2000,2000,3000, 1000, 3000])


# due_date =  
color = pd.Series(['B', 'B', 'B', 'B', 'B'])
demand = pd.DataFrame({'Size': size,'Quantity': quantity, 'Color': color})
deadline = [391600, 392400, 545438, 563333, 25111  ]

# Step 2 Input current mold and mold inventory

current_left = ['C7IJ', 'J2YZA1' ]
current_right = ['C7QRST', 'C5MNOP']

available_molds = df['Mold_name']


# Step 3: Generate possible combinations of molds to fulfill the demand

df2 = df[['Size', 'Number_of_pairs','Mold_name', 'Injection_volume', 'Cycle_Time']]

mold_sets = demand.merge(df2, how='left')

mold_sets = mold_sets.sort_values(['Injection_volume','Size'])


#mold_sets = mold_sets.sort_values(['Cycle_Time','Size'])
mold_sets

,Size,Quantity,Color,Number_of_pairs,Mold_name,Injection_volume,Cycle_Time
0,C4,2000,B,2,C4IJ,195.0,493
1,C4,2000,B,2,C4UV,197.0,503
2,C4,2000,B,2,C4K1L1,197.0,493
4,C5,2000,B,2,C5KL,218.0,494
5,C5,2000,B,2,C5YZ,220.0,504
6,C5,2000,B,2,C5Y1Z1,221.3,495
9,C7,3000,B,2,C7WX,252.0,507
8,C7,3000,B,2,C7IJ,252.2,467
10,C7,3000,B,2,C7I1J1,254.4,467
11,C7,3000,B,2,C7K3L3,255.0,507


In [232]:
# Identify number of cycles required to meet the demand of each size if one mold is used

df_master = pd.DataFrame(mold_sets.groupby('Size')[['Number_of_pairs', 'Quantity', 'Cycle_Time']].agg('mean'))
df_master['Max_cycle'] = df_master['Quantity']/df_master['Number_of_pairs']
df_master['Total_time'] = df_master['Cycle_Time']*df_master['Max_cycle']
df_master['No_molds'] = pd.DataFrame(mold_sets.groupby('Size')['Quantity'].agg('count'))
df_master


,Number_of_pairs,Quantity,Cycle_Time,Max_cycle,Total_time,No_molds
Size,,,,,,
C4,2.500000,2000.0,489.500000,800.000000,391600.000000,4
C5,2.500000,2000.0,490.500000,800.000000,392400.000000,4
C7,2.666667,3000.0,484.833333,1125.000000,545437.500000,6
C8,2.571429,3000.0,482.857143,1166.666667,563333.333333,7
J2,2.250000,1000.0,506.500000,444.444444,225111.111111,8


In [233]:
max_time = df_master['Total_time']
max_time

Size
C4    391600.000000
C5    392400.000000
C7    545437.500000
C8    563333.333333
J2    225111.111111
Name: Total_time, dtype: float64

In [234]:
condition_size =  mold_sets['Size'] == size[1]
mold_sets_filter_by_size = mold_sets[condition_size]
mold_sets_filter_by_size

,Size,Quantity,Color,Number_of_pairs,Mold_name,Injection_volume,Cycle_Time
4,C5,2000,B,2,C5KL,218.0,494
5,C5,2000,B,2,C5YZ,220.0,504
6,C5,2000,B,2,C5Y1Z1,221.3,495
7,C5,2000,B,4,C5MNOP,294.0,469


In [235]:
''' Validation for Deadline and set priority'''

mold_required_list = []
for i in range(len(size)): 
    condition_size =  mold_sets['Size'] == size[i]
    mold_sets_filter_by_size = mold_sets[condition_size]
    cumsum_product_demand = int(demand[demand['Size'] == size[i]]['Quantity']) - np.cumsum(mold_sets_filter_by_size['Number_of_pairs']*deadline[i]//mold_sets_filter_by_size['Cycle_Time'])

    mold_required = 0
    for j in range(len(cumsum_product_demand)):
        if (j == 0) and cumsum_product_demand.iloc[0] < 0:
            mold_required = 1

        
        if cumsum_product_demand.iloc[j] > 0:
            mold_required = j + 2


        if (j == len(cumsum_product_demand)-1) and cumsum_product_demand.iloc[-1] > 0:
            mold_required = 0
            print('Cannot produce size', size[i], 'on the given deadline')

        

    
    mold_required_list.append(mold_required)

mold_required_list

# Test case when not meet deadline -> ok

Cannot produce size C8 on the given deadline


[2, 2, 2, 1, 0]

In [236]:
''' Determine to use mold 3,4 '''


df_master['No_molds'] > mold_required_list

Size
C4    True
C5    True
C7    True
C8    True
J2    True
Name: No_molds, dtype: bool

In [246]:
int(1.5)

1

In [237]:
df_master_test = pd.DataFrame(mold_sets.groupby(['Size', 'Number_of_pairs'])[['Number_of_pairs', 'Quantity', 'Injection_volume', 'Cycle_Time']].agg('mean'))
df_master_test['No_molds'] = pd.DataFrame(mold_sets.groupby(['Size', 'Number_of_pairs'])['Quantity'].agg('count'))
df_master_test[df_master_test['Number_of_pairs'] == 2]['No_molds'] - mold_required_list

Size  Number_of_pairs
C4    2                  1
C5    2                  1
C7    2                  2
C8    2                  4
J2    2                  6
Name: No_molds, dtype: int64

In [238]:
mold_sets[mold_sets['Number_of_pairs'] == 2]

,Size,Quantity,Color,Number_of_pairs,Mold_name,Injection_volume,Cycle_Time
0,C4,2000,B,2,C4IJ,195.0,493
1,C4,2000,B,2,C4UV,197.0,503
2,C4,2000,B,2,C4K1L1,197.0,493
4,C5,2000,B,2,C5KL,218.0,494
5,C5,2000,B,2,C5YZ,220.0,504
6,C5,2000,B,2,C5Y1Z1,221.3,495
9,C7,3000,B,2,C7WX,252.0,507
8,C7,3000,B,2,C7IJ,252.2,467
10,C7,3000,B,2,C7I1J1,254.4,467
11,C7,3000,B,2,C7K3L3,255.0,507


In [239]:
condition_size =  mold_sets['Size'] == size[4]
mold_sets_filter_by_size = mold_sets[condition_size]
cumsum_product_demand = int(demand[demand['Size'] == size[4]]['Quantity']) - np.cumsum(mold_sets_filter_by_size['Number_of_pairs']*deadline[4]//mold_sets_filter_by_size['Cycle_Time'])
cumsum_product_demand

25    2893
22    2786
23    2686
26    2588
24    2481
28    2274
27    2067
dtype: int64

In [240]:
len(size)

5

In [241]:
mold_sets[condition_size]['Number_of_pairs']*mold_sets[condition_size]['Cycle_Time']



25     936
22     938
23     998
26    1018
24     938
28    1932
27    1932
dtype: int64

In [242]:
No_molds = []


def find_number_mold():

#(Other solvers: pywraplp.Solver.CreateSolver('SOLVERNAME'))
    op = pywraplp.Solver.CreateSolver('SCIP')
 
# Define decision variables
   
    x = []
    for i in range(len(df_master)):
        x.append(op.IntVar(0.0, op.infinity(), 'x'))
    # x0 = op.IntVar(0.0, op.infinity(), 'x') # 1st size
    # x1 = op.IntVar(0.0, op.infinity(), 'y') # 2nd size
    # x2 = op.IntVar(0.0, op.infinity(), 'y') # 3rd size
    # x3 = op.IntVar(0.0, op.infinity(), 'y') # 4th size

# Add objective function to the environment
    op.Maximize(sum(x))

# Add constraints to the environment
    op.Add(sum(x) <=16) # Constrain 1: max mold = 16 = 2* 8
    # op.Add(sum(x) >=8)
    for i in range(len(df_master)):
        # Constraint 2: Due time
        op.Add(1/df_master['Total_time'][i]*x[i] >= 3/max_time[i]) # Hard code 4 to get the results
        # Constraint 3: mold, size >= 1
        # op.Add(x[i] >=1)
        # Constraint 4: mold used <= molds available
        op.Add(x[i] <=df_master['No_molds'][i])
        # Constrain 3: equal number of shift

# Solve the problem 
    status = op.Solve()

    
# The status of the solution
    if status == pywraplp.Solver.OPTIMAL:
        print('Solution:')
        print('Objective value =', op.Objective().Value())
        for i in range(len(df_master)):
            print('Size', str(df_master.index[i]), 'No. Molds:', x[i].solution_value())
            No_molds.append(x[i].solution_value())
        # print('y =', x[1].solution_value())
        # print('z =', x[2].solution_value())
        # print('k =', x[3].solution_value())
    else:
        print('The problem does not have an optimal solution.')

    return(No_molds)
  
if __name__ == '__main__':
    find_number_mold()
    



Solution:
Objective value = 16.0
Size C4 No. Molds: 4.0
Size C5 No. Molds: 3.0
Size C7 No. Molds: 3.0
Size C8 No. Molds: 3.0
Size J2 No. Molds: 3.0


In [243]:
No_molds

[4.0, 3.0, 3.0, 3.0, 3.0]

In [244]:
df_master.index[0]

'C4'

In [245]:
# Select 1st combination mold

Mold_combination = mold_sets[mold_sets['Size'] ==  df_master.index[0]].head(int(No_molds[0]))


for i in range(1,len(df_master)):
    result = mold_sets[mold_sets['Size'] ==  df_master.index[i]].head(int(No_molds[i]))
    Mold_combination = pd.concat([Mold_combination, result], ignore_index= True)

Mold_combination

,Size,Quantity,Color,Number_of_pairs,Mold_name,Injection_volume,Cycle_Time
0,C4,2000,B,2,C4IJ,195.0,493
1,C4,2000,B,2,C4UV,197.0,503
2,C4,2000,B,2,C4K1L1,197.0,493
3,C4,2000,B,4,C4MNOP,292.7,469
4,C5,2000,B,2,C5KL,218.0,494
5,C5,2000,B,2,C5YZ,220.0,504
6,C5,2000,B,2,C5Y1Z1,221.3,495
7,C7,3000,B,2,C7WX,252.0,507
8,C7,3000,B,2,C7IJ,252.2,467
9,C7,3000,B,2,C7I1J1,254.4,467
